In [12]:
from numpy import zeros, int8, log
import pylab
import sys
import re
import time
import codecs

In [4]:
def initializeParameters(n, m, k, lamda, theta):
    for i in range(0, n):
        normalization = sum(lamda[i, :])
        for j in range(0, k):
            lamda[i, j] /= normalization

    for i in range(0, k):
        normalization = sum(theta[i, :])
        for j in range(0, m):
            theta[i, j] /= normalization


In [249]:

def do_e_step(n, m, k, lamda, theta, p):
    for i in range(0, n):
        for j in range(0, m):
            denominator = 0
            for ik in range(0, k):
                p[i, j, ik] = theta[ik, j] * lamda[i, ik]
                denominator += p[i, j, ik]
            if denominator == 0:
                for ik in range(0, k):
                    p[i, j, ik] = 0
            else:
                for ik in range(0, k):
                    p[i, j, ik] /= denominator

In [250]:
def do_m_step(n, m, k, lamda, theta, p, docs):
    # update theta
    for ik in range(0, k):
        denominator = 0
        for j in range(0, m):
            theta[ik, j] = 0
            for i in range(0, n):
                theta[ik, j] += docs[i, j] * p[i, j, ik]
            denominator += theta[ik, j]
        if denominator == 0:
            for j in range(0, m):
                theta[ik, j] = 1.0 / m
        else:
            for j in range(0, m):
                theta[ik, j] /= denominator
    # update lamda
    for i in range(0, n):
        for ik in range(0, k):
            lamda[i, ik] = 0
            denominator = 0
            for j in range(0, m):
                lamda[i, ik] += docs[i, j] * p[i, j, ik]
                denominator += docs[i, j]
            if denominator == 0:
                lamda[i, ik] = 1.0 / k
            else:
                lamda[i, ik] /= denominator

In [8]:
# calculate the log likelihood
def LogLikelihood(n, m, k, lamda, theta, docs):
    loglikelihood = 0
    for i in range(0, n):
        for j in range(0, m):
            tmp = 0
            for ik in range(0, k):
                tmp += theta[ik, j] * lamda[i, ik]
            if tmp > 0:
                loglikelihood += docs[i, j] * log(tmp)
    return loglikelihood

In [43]:
import gzip
import json
import numpy as np
from collections import defaultdict

workpath="/data/jmoreno/Datasets/signalmedia/"

f=gzip.open(workpath+'signalmedia-1k.jsonl.gz','rb')


docs=[json.loads(s.decode('utf-8')) for s in f.readlines()]
titres=[x['title'] for x in docs]
content=[x['content'] for x in docs]
print(len(titres))

def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent


cleantitres=[sentense2cleanTokens(x) for x in content]

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=5, stop_words='english')
tfidfsk = tfidf_vectorizer.fit_transform(cleantitres)


1000


In [251]:
def PLSA(docs, k, niter=10, threshold=10.0):
    n = len(docs)
    m = docs[0].size
    # lamda[i, j] : p(zj|di)
    lamda = pylab.random([n, n_topics])

    # theta[i, j] : p(wj|zi)
    theta = pylab.random([n_topics, m])

    # p[i, j, k] : p(zk|di,wj)
    p = zeros([n, m, n_topics])
    initializeParameters(n,m,k,lamda,theta)

    # EM algorithm
    oldLoglikelihood = 1
    newLoglikelihood = 1
    for i in range(niter):
        do_e_step(n, m, k, lamda, theta, p)
        do_m_step(n, m, k, lamda, theta, p, docs)
        newLoglikelihood = LogLikelihood(n, m, k, lamda, theta, docs)
        print("Iter=",i+1, "LogLikelihood=", str(newLoglikelihood))
        if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
            break
        oldLoglikelihood = newLoglikelihood
    return lamda,theta,p

In [252]:
n_topics=10

lamda,theta,p = PLSA(tfidfsk.todense(),n_topics)

Iter= 1 LogLikelihood= -62265.6522015
Iter= 2 LogLikelihood= -62044.223311
Iter= 3 LogLikelihood= -61791.6190231
Iter= 4 LogLikelihood= -61488.5529111
Iter= 5 LogLikelihood= -61144.7722196
Iter= 6 LogLikelihood= -60787.6404451
Iter= 7 LogLikelihood= -60445.5141423
Iter= 8 LogLikelihood= -60136.0522363
Iter= 9 LogLikelihood= -59865.7475652
Iter= 10 LogLikelihood= -59634.0689689


In [50]:
ivocab = tfidf_vectorizer.get_feature_names()
clusters=defaultdict(list)
for i, doclamda in enumerate(lamda):
    clusters[np.argmax(doclamda)].append(i)
for i in range(len(theta)):
    print("\n-----\nCluster label:"," ".join([ivocab[x] for x in np.argsort(theta[i])[::-1][:10]]))
    print([titres[x] for x in clusters[i][:10]])


-----
Cluster label: game said season people year city think court appeared home
['2015’s Opening Game Predictions', 'Police chiefs warn ministers Were too broke to go on the beat', 'PINK SLAMS DEMI?!', 'NHS patients to be given option of travelling to Calais for surgical procedures', 'Caterpillar Still On Hook For $4.5M In Wash. Asbestos Suit', "Facebook ready to test button that goes beyond 'like'", "Berra's best World Series unrewarded", 'How Did Trump Do in the Debate?', 'UPDATE 1-German president warns of limits to number of refugees', 'Dancing On Ice skater rapist Joseph Tsang captured in Hong Kong honey trap']

-----
Cluster label: said india use mr garden year add management government program
['Worcester breakfast club for veterans gives hunger its marching orders', 'Hockey mum on multinational tax revenue', 'The Streaming Media Device Landscape', 'Nursery celebrates six years since its opening', 'West Brom turn down another Tottenham bid for Saido Berahino', 'Christmas holid

In [52]:
lamda,theta,p = PLSA(tfidfsk.todense(),n_topics, niter=50)

clusters=defaultdict(list)
for i, doclamda in enumerate(lamda):
    clusters[np.argmax(doclamda)].append(i)
for i in range(len(theta)):
    print("\n-----\nCluster label:"," ".join([ivocab[x] for x in np.argsort(theta[i])[::-1][:10]]))
    print([titres[x] for x in clusters[i][:10]])

Iter= 1 LogLikelihood= -62261.3779017
Iter= 2 LogLikelihood= -62033.8794534
Iter= 3 LogLikelihood= -61773.7125932
Iter= 4 LogLikelihood= -61461.573419
Iter= 5 LogLikelihood= -61108.696449
Iter= 6 LogLikelihood= -60745.0348994
Iter= 7 LogLikelihood= -60400.5513823
Iter= 8 LogLikelihood= -60091.8081019
Iter= 9 LogLikelihood= -59821.7859213
Iter= 10 LogLikelihood= -59587.5248803
Iter= 11 LogLikelihood= -59384.9115852
Iter= 12 LogLikelihood= -59209.9188246
Iter= 13 LogLikelihood= -59058.8993532
Iter= 14 LogLikelihood= -58928.5888498
Iter= 15 LogLikelihood= -58815.4514743
Iter= 16 LogLikelihood= -58716.3084387
Iter= 17 LogLikelihood= -58629.132674
Iter= 18 LogLikelihood= -58552.0355115
Iter= 19 LogLikelihood= -58483.1346166
Iter= 20 LogLikelihood= -58421.3078934
Iter= 21 LogLikelihood= -58365.6086048
Iter= 22 LogLikelihood= -58314.5636064
Iter= 23 LogLikelihood= -58267.5401251
Iter= 24 LogLikelihood= -58224.260806
Iter= 25 LogLikelihood= -58184.0378343
Iter= 26 LogLikelihood= -58146.5246636

In [151]:
from scipy.special import gammaln, psi

def dirichlet_expectation(alpha):
    """
    For a vector theta ~ Dir(alpha), computes E[log(theta)] given alpha.
    """
    if (len(alpha.shape) == 1):
        return(psi(alpha) - psi(np.sum(alpha)))
    return(psi(alpha) - psi(np.sum(alpha, 1))[:, np.newaxis])

class OnlineLDA:
    """
    Implements online VB for LDA as described in (Hoffman et al. 2010).
    """

    def __init__(self, vocab, K, D, alpha, eta, tau0=1024.0, kappa=0.7):
        """
        Arguments:
        K: Number of topics
        vocab: A set of words to recognize. When analyzing documents, any word
           not in this set will be ignored.
        D: Total number of documents in the population. For a fixed corpus,
           this is the size of the corpus. In the truly online setting, this
           can be an estimate of the maximum number of documents that
           could ever be seen.
        alpha: Hyperparameter for prior on weight vectors theta
        eta: Hyperparameter for prior on topics beta
        tau0: A (positive) learning parameter that downweights early iterations
        kappa: Learning rate: exponential decay rate---should be between
             (0.5, 1.0] to guarantee asymptotic convergence.

        Note that if you pass the same set of D documents in every time and
        set kappa=0 this class can also be used to do batch VB.
        """
        self._vocab = dict()
        for word in vocab:
            self._vocab[word] = len(self._vocab)

        self._K = K
        self._W = len(self._vocab)
        self._D = D
        self._alpha = alpha
        self._eta = eta
        self._tau0 = tau0 + 1
        self._kappa = kappa
        self._updatect = 0

        # Initialize the variational distribution q(beta|lambda)
        self._lambda = 1*np.random.gamma(100., 1./100., (self._K, self._W))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = np.exp(self._Elogbeta)

    def do_e_step(self, wordids, wordcts):
        meanchangethresh = 0.001
        batchD = len(wordids)

        # Initialize the variational distribution q(theta|gamma) for
        # the mini-batch
        gamma = 1*np.random.gamma(100., 1./100., (batchD, self._K))
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = np.exp(Elogtheta)

        sstats = np.zeros(self._lambda.shape)
        # Now, for each document d update that document's gamma and phi
        it = 0
        meanchange = 0
        for d in range(0, batchD):
            # print(sum(wordcts[d]))
            # These are mostly just shorthand (but might help cache locality)
            ids = wordids[d]
            cts = wordcts[d]
            gammad = gamma[d, :]
            Elogthetad = Elogtheta[d, :]
            expElogthetad = expElogtheta[d, :]
            expElogbetad = self._expElogbeta[:, ids]
            # The optimal phi_{dwk} is proportional to 
            # expElogthetad_k * expElogbetad_w. phinorm is the normalizer.
            phinorm = np.dot(expElogthetad, expElogbetad) + 1e-100
            # Iterate between gamma and phi until convergence
            for it in range(0, 100):
                lastgamma = gammad
                # We represent phi implicitly to save memory and time.
                # Substituting the value of the optimal phi back into
                # the update for gamma gives this update. Cf. Lee&Seung 2001.
                gammad = self._alpha + expElogthetad * \
                    np.dot(cts / phinorm, expElogbetad.T)
                # print(gammad[:, np.newaxis])
                Elogthetad = dirichlet_expectation(gammad)
                expElogthetad = np.exp(Elogthetad)
                phinorm = np.dot(expElogthetad, expElogbetad) + 1e-100
                # If gamma hasn't changed much, we're done.
                meanchange = np.mean(abs(gammad - lastgamma))
                if (meanchange < meanchangethresh):
                    break
            gamma[d, :] = gammad
            # Contribution of document d to the expected sufficient
            # statistics for the M step.
            sstats[:, ids] += np.outer(expElogthetad.T, cts/phinorm)

        # This step finishes computing the sufficient statistics for the
        # M step, so that
        # sstats[k, w] = \sum_d n_{dw} * phi_{dwk} 
        # = \sum_d n_{dw} * exp{Elogtheta_{dk} + Elogbeta_{kw}} / phinorm_{dw}.
        sstats = sstats * self._expElogbeta

        return((gamma, sstats))

    def update_lambda(self, wordids, wordcts):
        """
        First does an E step on the mini-batch given in wordids and
        wordcts, then uses the result of that E step to update the
        variational parameter matrix lambda.

        Arguments:
        docs:  List of D documents. Each document must be represented
               as a string. (Word order is unimportant.) Any
               words not in the vocabulary will be ignored.

        Returns gamma, the parameters to the variational distribution
        over the topic weights theta for the documents analyzed in this
        update.

        Also returns an estimate of the variational bound for the
        entire corpus for the OLD setting of lambda based on the
        documents passed in. This can be used as a (possibly very
        noisy) estimate of held-out likelihood.
        """

        # rhot will be between 0 and 1, and says how much to weight
        # the information we got from this mini-batch.
        rhot = pow(self._tau0 + self._updatect, -self._kappa)
        self._rhot = rhot
        # Do an E step to update gamma, phi | lambda for this
        # mini-batch. This also returns the information about phi that
        # we need to update lambda.
        (gamma, sstats) = self.do_e_step(wordids, wordcts)
        # Estimate held-out likelihood for current values of lambda.
        bound = self.approx_bound(wordids, wordcts, gamma)
        # Update lambda based on documents.
        self._lambda = self._lambda * (1-rhot) + \
            rhot * (self._eta + self._D * sstats / len(wordids))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = np.exp(self._Elogbeta)
        self._updatect += 1

        return gamma,self._lambda,bound

    def approx_bound(self, wordids, wordcts, gamma):
        """
        Estimates the variational bound over *all documents* using only
        the documents passed in as "docs." gamma is the set of parameters
        to the variational distribution q(theta) corresponding to the
        set of documents passed in.

        The output of this function is going to be noisy, but can be
        useful for assessing convergence.
        """

        # This is to handle the case where someone just hands us a single
        # document, not in a list.
        batchD = len(wordids)

        score = 0
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = np.exp(Elogtheta)

        # E[log p(docs | theta, beta)]
        for d in range(0, batchD):
            gammad = gamma[d, :]
            ids = wordids[d]
            cts = np.array(wordcts[d])
            phinorm = np.zeros(len(ids))
            for i in range(0, len(ids)):
                temp = Elogtheta[d, :] + self._Elogbeta[:, ids[i]]
                tmax = max(temp)
                phinorm[i] = np.log(sum(np.exp(temp - tmax))) + tmax
            score += np.sum(cts * phinorm)

        # E[log p(theta | alpha) - log q(theta | gamma)]
        score += np.sum((self._alpha - gamma)*Elogtheta)
        score += np.sum(gammaln(gamma) - gammaln(self._alpha))
        score += sum(gammaln(self._alpha*self._K) - gammaln(np.sum(gamma, 1)))

        # Compensate for the subsampling of the population of documents
        score = score * self._D / len(wordids)

        # E[log p(beta | eta) - log q (beta | lambda)]
        score = score + np.sum((self._eta-self._lambda)*self._Elogbeta)
        score = score + np.sum(gammaln(self._lambda) - gammaln(self._eta))
        score = score + np.sum(gammaln(self._eta*self._W) - 
                              gammaln(np.sum(self._lambda, 1)))

        return(score)

In [161]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(min_df=5, stop_words='english')
tfsk = tf_vectorizer.fit_transform(cleantitres)
ivocabtf = tf_vectorizer.get_feature_names()

In [162]:
ndocs = len(tfsk.todense())
n_topics = 10
olda = OnlineLDA(ivocabtf, n_topics, ndocs, 1./n_topics, 1./n_topics)
tfindices = [list(x.indices) for x in tfsk]
tfdata = [list(x.data) for x in tfsk]

In [163]:
gamma,_lambda,bound = olda.update_lambda(tfindices,tfdata)
print(bound)

-1661607.7314


In [164]:
clusters=defaultdict(list)
for i, docgamma in enumerate(gamma):
    clusters[np.argmax(docgamma)].append(i)
for i in range(len(_lambda)):
    print("\n-----\nCluster label:"," ".join([ivocabtf[x] for x in np.argsort(_lambda[i])[::-1][:10]]))
    print([titres[x] for x in clusters[i][:10]])


-----
Cluster label: said com city jewish th time place technology business event
['Worcester breakfast club for veterans gives hunger its marching orders', 'Pay up or face legal action: DBKL', 'Meredith and Walt Disney are big market movers', "America's Most Expensive Rental Markets", 'Local Happenings: Danville\'s Village Theatre presenting "Streetcar Named Desire" stage performance, film screening', "Virgin America Offers Free Uber Rides With Every Dallas Love Field Flight* And Brings 'Friday Night Flights' To The Big D", 'Student Rocketeers Wanted for World’s Largest Rocket Competition 2015-2016', '20-Sep-2015 02:19', 'Shell says it will cease Alaska Arctic exploration, cites disappointing outcome at well', 'Martín Cáceres suspended by Juventus for alleged drink-driving incident']

-----
Cluster label: said new year people com september like national www way
['This New Dating App Will Ruin Your Internet Game', 'Harwood Feffer LLP Announces Investigation of VASCO Data Security Inte

In [207]:
import scipy.sparse as sp
from sys import float_info


class NMF(object):
    def __init__(self, A, k=5):
        n, m = A.shape
        W0 = np.random.rand(n, k)
        H0 = np.random.rand(k, m)

        self._errors = []
        self._A = A
        self._H = H0
        self._W = W0
        
    def setWH(self, W, H):
        self._W = W
        self._H = H


    def run(self, niter=100, calc_error_num=10):
        H = self._H
        W = self._W
        eps = float_info.min
        A = self._A
        for i in range(niter):

            # update H
            # H=H.*(W'*A)./(W'*W*H+eps)
            H = H * ((A.T * W).T) / (W.T.dot(W).dot(H) + eps)
            # update W
            # W=W.*(A*H')./(W*(H*H')+eps);
            W = W * (A * H.T) / (W.dot(H.dot(H.T)) + eps)
            if (i+1)%calc_error_num==0:
                M = A - W.dot(H)
                error = np.multiply(M, M).sum()
                self._errors.append(error)
                print("Iter=",i+1, "Error=", str(error))
        self._H = H
        self._W = W
        return W,H

In [237]:
mynmf = NMF(tfidfsk, n_topics)
W,H = mynmf.run()

Iter= 10 Error= 944.393064172
Iter= 20 Error= 935.621659477
Iter= 30 Error= 933.285336116
Iter= 40 Error= 932.121833161
Iter= 50 Error= 931.670368424
Iter= 60 Error= 931.4912954
Iter= 70 Error= 931.39852162
Iter= 80 Error= 931.336801744
Iter= 90 Error= 931.293238229
Iter= 100 Error= 931.26190672


In [244]:
def nndsvd(A, n_topics, min_val=0.01):
    U, s, V = np.linalg.svd(A.todense(), full_matrices=True)
    print(U.shape, s.shape, V.shape)
    n, m = A.shape
    W = np.random.rand(n, n_topics)
    H = np.random.rand(n_topics, m)
    #choose the first singular triplet to be nonnegative
    W[:,0] = np.sqrt(s[1]) * abs(np.ravel(U[:,1]))
    H[0,:] = np.sqrt(s[1]) * abs(V[:,1].T)

    #2nd SVD for the other factors
    for i in range(1,n_topics):
        uu = np.ravel(U[:,i])
        vv = np.ravel(V[:,i])
        uup = uu.clip(min=0)
        uun = abs(uu.clip(max=0))
        vvp = vv.clip(min=0)
        vvn = abs(vv.clip(max=0))
        n_uup = np.linalg.norm(uup)
        n_vvp = np.linalg.norm(vvp)
        n_uun = np.linalg.norm(uun)
        n_vvn = np.linalg.norm(vvn)
        termp = n_uup*n_vvp
        termn = n_uun*n_vvn
        if termp >= termn:
            W[:,i] = np.sqrt(s[i]*termp)*uup/n_uup
            H[i,:] = np.sqrt(s[i]*termp)*vvp.T/n_vvp
        else:
            W[:,i] = np.sqrt(s[i]*termn)*uun/n_uun
            H[i,:] = np.sqrt(s[i]*termn)*vvn.T/n_vvn
    W[W==0]=min_val
    H[H==0]=min_val
    return W,H

In [245]:
W0,H0 = nndsvd(tfidfsk,10)

(1000, 1000) (1000,) (5739, 5739)


In [246]:
W0

array([[ 0.03833935,  0.01      ,  0.04154105, ...,  0.01      ,
         0.00535324,  0.00231954],
       [ 0.11288261,  0.12679343,  0.01      , ...,  0.07377825,
         0.01      ,  0.01113601],
       [ 0.02493787,  0.01      ,  0.01      , ...,  0.01      ,
         0.01162546,  0.0222418 ],
       ..., 
       [ 0.02226615,  0.02501006,  0.01      , ...,  0.01      ,
         0.04444898,  0.01      ],
       [ 0.07489758,  0.08412741,  0.01      , ...,  0.01      ,
         0.0809301 ,  0.01      ],
       [ 0.06502651,  0.01      ,  0.01      , ...,  0.0223106 ,
         0.00992611,  0.01      ]])

In [247]:
mynmf.setWH(W0,H0)
Wplus,Hplus = mynmf.run()


Iter= 10 Error= 934.425135364
Iter= 20 Error= 932.193831452
Iter= 30 Error= 931.541903653
Iter= 40 Error= 931.30711191
Iter= 50 Error= 931.196957901
Iter= 60 Error= 931.137488254
Iter= 70 Error= 931.099977679
Iter= 80 Error= 931.073128382
Iter= 90 Error= 931.050360681
Iter= 100 Error= 931.033087171


In [248]:
clusters=defaultdict(list)
for i, docw in enumerate(Wplus):
    clusters[np.argmax(docw)].append(i)
for i in range(len(Hplus)):
    print("\n-----\nCluster label:"," ".join([ivocab[x] for x in np.argsort(Hplus[i])[::-1][:10]]))
    print([titres[x] for x in clusters[i][:10]])


-----
Cluster label: season team game year league games cup said win rugby
['Worcester breakfast club for veterans gives hunger its marching orders', 'The Return Of The Nike Air Max Sensation Has 80’s Babies Hyped!', 'Former pitcher Andujar dies at 62', "Bayern Munich's Arjen Robben Sidelined With Groin Injury", "Corey Seager's days as the Dodgers' starting shortstop are numbered", 'Watkins the star in Wormelow awards evening', 'City welcomes Scotland before Rugby World Cup kicks off tomorrow', "Berra's best World Series unrewarded", 'TALKING TACTICS: finding the right Plan B', "Spennymoor Town must be wary of 'full of confidence' Clitheroe, says Jason Ainsley"]

-----
Cluster label: company business data market information services solutions technology research products
['Jumpshot Gives Marketers Renewed Visibility Into Paid and Organic Keywords With Launch of Jumpshot Elite', 'Semtech (SMTC) Trading Near $17.29 Resistance Level', "It's a Mad world at the Melbourne Bar, as long as yo